In [7]:
# import notebook_importer

import numpy as np
import pickle
import random
import zlib
import torch
import torch.nn as nn
from torch.autograd import Variable


In [62]:
BASE**(PRECISION * Q_MAXDEGREE)

4294967296

In [100]:
BASE = 2

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL
BOUND = BASE**PRECISION

# Q field
# Q = 6497992661811505123 # < 64 bits
Q = 2147483647 # < 32 bits
Q_MAXDEGREE = 2
# assert Q > BASE**(PRECISION * Q_MAXDEGREE) # supported multiplication degree (without truncation)


def encode(rational, field=Q, precision_fractional=PRECISION_FRACTIONAL):
    upscaled = rational * BASE**precision_fractional
    field_element = upscaled % field
    return field_element

def decode(field_element, field=Q, precision_fractional=PRECISION_FRACTIONAL):
    mask = (field_element <= int(field/2)).double()
    upscaled = (field_element * mask) + ((field_element - field) * (1 - mask))
    rational = upscaled / BASE**precision_fractional
    return rational

def share(secret, field=Q):
    first  = torch.rand(4,4).double() * field
    second = (secret - first) % field
    return [first, second]

def reconstruct(shares, field=Q):
    s = sum(shares) % field
    return s

def reconstruct_data(shares,field=Q):
    return sum(shares) % field

a = torch.rand(4,4).double()
a_enc = encode(a).floor()
a_sh = share(a_enc)

b = torch.rand(4,4).double()
b_enc = encode(b).floor()

c_sh = (b_enc.mm(a_sh[0]),b_enc.mm(a_sh[1]))
c_enc_enc = reconstruct(c_sh)
c_enc = decode(c_enc_enc)
c = decode(c_enc)

# compute loss
target = torch.eye(4).double()
grad = ((c - target))/8

# begin backprop
grad_enc = encode(grad).floor()

b_enc_grad1 = grad_enc.mm(a_sh[0].transpose(0,1))
b_enc_grad2 = grad_enc.mm(a_sh[1].transpose(0,1))

decode(decode(reconstruct_data([b_enc_grad1,b_enc_grad2])))


 0.2625  0.2651  0.3413  0.2708
 0.3527  0.2913  0.3530  0.4402
 0.2449  0.3435  0.4886  0.4565
 0.1048  0.1776  0.1880  0.2262
[torch.DoubleTensor of size 4x4]

In [93]:
# a = torch.rand(4,4).double()
# b = torch.rand(4,4).double()

a = Variable(a,requires_grad=True)
b = Variable(b,requires_grad=True)
c = b.mm(a)
loss = nn.MSELoss()(c,Variable(target))
loss.backward()

b.grad

Variable containing:
 0.0412 -0.0065  0.0238  0.0395
 0.2581  0.1835  0.3691  0.1992
 0.3841  0.3004  0.5210  0.3286
 0.4906  0.3286  0.5303  0.2489
[torch.DoubleTensor of size 4x4]

In [44]:
a.grad

Variable containing:
 0.2223  0.1426  0.2775  0.3583
 0.1103  0.0858  0.1361  0.0889
 0.2401  0.1669  0.2958  0.3861
 0.3302  0.2310  0.3413  0.4243
[torch.DoubleTensor of size 4x4]

In [ ]:
# Inherit from Function
class LinearFunction(Function):

    # Note that both forward and backward are @staticmethods
    @staticmethod
    # bias is an optional argument
    def forward(ctx, input, weight, bias=None):
        ctx.save_for_backward(input, weight, bias)
        output = input.mm(weight.t())
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output

    # This function has only a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx, grad_output):
        # This is a pattern that is very convenient - at the top of backward
        # unpack saved_tensors and initialize all gradients w.r.t. inputs to
        # None. Thanks to the fact that additional trailing Nones are
        # ignored, the return statement is simple even when the function has
        # optional inputs.
        input, weight, bias = ctx.saved_variables
        grad_input = grad_weight = grad_bias = None

        # These needs_input_grad checks are optional and there only to
        # improve efficiency. If you want to make your code simpler, you can
        # skip them. Returning gradients for inputs that don't require it is
        # not an error.
        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0).squeeze(0)

        return grad_input, grad_weight, grad_bias

In [1641]:

# class SharedLinear(nn.Module):
    
#     def __init__(self,input_dim,output_dim):
        
#         self.weights = Vtorch.rand(4,4).double()

In [1642]:
# class Tensor():
    
#     def __init__(self,data=torch.rand(4,4).double()):
#         if(type(data) == Variable):
#             self.data = data
#             self.data.data = self.data.data.double()
#         else:
#             self.data = Variable(data.double())
    
#     def encode(self):
#         self.data = encode(self.data)
#         return self
    
#     def decode(self):
#         self.data = decode(self.data)
#         return self
    
#     def share(self):
#         first,second = share(self.data)
        
#     def dot(self,y):
#         return Tensor(data=self.data.mm(y.data))
    
#     def __repr__(self):
#         return str(self.data)
    
# def SharedTensor():
    
#     def __init__(self,shares):
#         self.shares = shares
    
#     def dot(self,y):
        

In [1643]:
a = Tensor().encode()
b = Tensor()


In [1644]:
a.dot(b).decode()

Variable containing:
 1.4504  1.1138  1.6164  0.9516
 1.2566  1.0230  1.2962  1.2076
 1.4686  1.0174  1.7135  1.4413
 1.3000  0.7191  1.1913  1.3451
[torch.DoubleTensor of size 4x4]

Variable containing:
 0.1049  0.1783  0.1363  0.1678
 0.1887  0.3355  0.2936  0.2936
 0.1678  0.2307  0.1678  0.2726
 0.0000  0.0000  0.0000  0.0000
[torch.DoubleTensor of size 4x4]

Variable containing:
 0.2427  0.2537  0.2673  0.2825
 0.1318  0.2385  0.1412  0.2352
 0.2361  0.2097  0.2784  0.3678
 0.2019  0.2418  0.1410  0.2400
[torch.DoubleTensor of size 4x4]

In [1510]:
a.grad

Variable containing:
 0.3688  0.3187  0.3516  0.2178
 0.4972  0.3987  0.3940  0.2871
 0.4098  0.3410  0.2343  0.2304
 0.4251  0.4295  0.2830  0.2987
[torch.DoubleTensor of size 4x4]

In [1511]:
((c - target)/8)

Variable containing:
 0.1070  0.1845  0.1619  0.1203
 0.1533  0.0194  0.1543  0.0824
 0.2112  0.1755  0.0234  0.1674
 0.1539  0.1474  0.1336 -0.0131
[torch.DoubleTensor of size 4x4]

In [1481]:
((c - target)/8).mm(a.transpose(0,1))

Variable containing:
 0.2740  0.3677  0.2537  0.2359
 0.1600  0.3115  0.2594  0.1827
 0.2715  0.3490  0.2092  0.1978
 0.1562  0.3345  0.2068  0.1228
[torch.DoubleTensor of size 4x4]